<a href="https://colab.research.google.com/github/Khronophobia/NoName-BEE2-Addon/blob/master/comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ComfyUI Colab

This notebook runs [ComfyUI.](https://github.com/comfyanonymous/ComfyUI)

(WORK IN PROGRESS)

In [ ]:
#@title Install Dependencies
#@markdown ##### Clone ComfyUI from GitHub and install required dependencies.
!apt-get -y install -qq aria2

!git clone https://github.com/comfyanonymous/ComfyUI.git
webui_dir = "/content/ComfyUI"
%cd {webui_dir}
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

%cd {webui_dir}/custom_nodes
!wget -qq --show-progress -O advanced_model_merging.py https://github.com/comfyanonymous/ComfyUI_experiments/raw/master/advanced_model_merging.py
!wget -qq --show-progress -O reference_only.py https://github.com/comfyanonymous/ComfyUI_experiments/raw/master/reference_only.py
!wget -qq --show-progress -O sampler_rescalecfg.py https://github.com/comfyanonymous/ComfyUI_experiments/raw/master/sampler_rescalecfg.py
!wget -qq --show-progress -O sampler_tonemap.py https://github.com/comfyanonymous/ComfyUI_experiments/raw/master/sampler_tonemap.py
!wget -qq --show-progress -O sdxl_model_merging.py https://github.com/comfyanonymous/ComfyUI_experiments/raw/master/sdxl_model_merging.py

%cd {webui_dir}/models/vae_approx
!wget -qq --show-progress -O taesd_decoder.pth https://github.com/madebyollin/taesd/raw/main/taesd_decoder.pth
!wget -qq --show-progress -O taesdxl_decoder.pth https://github.com/madebyollin/taesd/raw/main/taesdxl_decoder.pth

In [ ]:
#@title ## (Optional) Mount Google Drive
#@markdown ##### Link this Colab to Google Drive and save your outputs there
output_path = "SD_WEBUI" #@param {type:"string"}
#@markdown ##### Where outputs will be saved (Can be the same as my A1111 colab).

import glob
import os
from google.colab import drive
drive.mount('/content/drive')
output_path = '/content/drive/MyDrive/' + output_path
# make necessary directories if not exists
!mkdir -p {output_path}/comfyui_output
!mkdir -p {output_path}/models
!mkdir -p {output_path}/upscalers/ESRGAN
!mkdir -p {output_path}/upscalers/SwinIR
!mkdir -p {output_path}/hypernetworks
!mkdir -p {output_path}/embeddings
!mkdir -p {output_path}/Lora

def linkDrivetoWebUI():
  %cd {webui_dir}

  !rm -rf output
  !ln -s {output_path}/comfyui_output {webui_dir}/output

  # Embeddings in drive
  %cd {webui_dir}/models
  !rm -rf embeddings
  !ln -s {output_path}/embeddings

  # Lora in drive
  %cd {webui_dir}/models
  !rm -rf loras
  !ln -s {output_path}/Lora {webui_dir}/models/loras

  # Hypernetworks in drive
  %cd {webui_dir}/models
  !rm -rf hypernetworks
  !ln -s {output_path}/hypernetworks

  # link all upscalers in the model folder
  %cd {webui_dir}/models/upscale_models
  upscalers_in_google_drive = glob.glob(f"{output_path}/upscalers/**/*.*", recursive=True)
  upscalers_in_google_drive_relpath = [os.path.relpath(upscaler, f"{output_path}/upscalers") for upscaler in upscalers_in_google_drive]
  print(f"Upscalers in Google Drive: {upscalers_in_google_drive_relpath}")
  for f in upscalers_in_google_drive:
    !ln -s {f}

linkDrivetoWebUI()

In [ ]:
#@title ## Configure Checkpoints
#@markdown Run this cell and follow further instructions
import ipywidgets as widgets
from IPython.display import display

## Model checkboxes
# SD 1.5
m_sd15 = widgets.Checkbox(value=False, description='Stable Diffusion 1.5')
m_sd15inpaint = widgets.Checkbox(value=False, description='Stable Diffusion 1.5 (inpainting)')
m_realv13 = widgets.Checkbox(value=False, description='Realistic Vision v1.3')
m_realv2 = widgets.Checkbox(value=False, description='Realistic Vision v2.0')
m_chilloutmix = widgets.Checkbox(value=False, description='Deliberate')
m_deliberate = widgets.Checkbox(value=False, description='ChilloutMix')
m_dreamshaperv3 = widgets.Checkbox(value=False, description='Dreamshaper V3')
m_dreamshaperv4 = widgets.Checkbox(value=False, description='Dreamshaper V4')
m_dreamshaperv5 = widgets.Checkbox(value=False, description='Dreamshaper V5')
m_ned = widgets.Checkbox(value=False, description='NeverEnding Dream')
m_openjourney = widgets.Checkbox(value=False, description='OpenJourney v4')
m_rev = widgets.Checkbox(value=False, description='ReV Animated')
m_perfectworld = widgets.Checkbox(value=False, description='Perfect World')
m_wd13 = widgets.Checkbox(value=False, description='Waifu Diffusion 1.3')
m_anythingv3 = widgets.Checkbox(value=False, description='Anything V3')
m_anythingv4 = widgets.Checkbox(value=False, description='Anything V4.5 (unofficial)')
m_anythingv5 = widgets.Checkbox(value=False, description='Anything V5')
m_counterfeitv2 = widgets.Checkbox(value=False, description='Counterfeit V2.5')
m_counterfeitv3 = widgets.Checkbox(value=False, description='Counterfeit V3.0')
m_aom2 = widgets.Checkbox(value=False, description='AbyssOrangeMix 2 (SFW)')
m_aom3 = widgets.Checkbox(value=False, description='AbyssOrangeMix 3')
m_aom3a1 = widgets.Checkbox(value=False, description='AbyssOrangeMix 3A1')
m_aom3a1b = widgets.Checkbox(value=False, description='AbyssOrangeMix 3A1B')
m_aom3a2 = widgets.Checkbox(value=False, description='AbyssOrangeMix 3A2')
m_aom3a3 = widgets.Checkbox(value=False, description='AbyssOrangeMix 3A3')
m_bom = widgets.Checkbox(value=False, description='BloodOrangeMix')
m_midnightmelt = widgets.Checkbox(value=False, description='Midnight Melt')
m_midnightmaple = widgets.Checkbox(value=False, description='Midnight Maple')
m_midnightmixermelt = widgets.Checkbox(value=False, description='Midnight Mixer Melt')
m_cornflowerv9 = widgets.Checkbox(value=False, description='Cornflower V9')
m_cornflowerx = widgets.Checkbox(value=False, description='Cornflower X')
m_ambientmix = widgets.Checkbox(value=False, description='AmbientMix')
m_hassaku = widgets.Checkbox(value=False, description='Hassaku')
m_grapefruit = widgets.Checkbox(value=False, description='Grapefruit')
m_meinamix = widgets.Checkbox(value=False, description='MeinaMix')
m_meinahentai = widgets.Checkbox(value=False, description='MeinaHentai')
m_nai = widgets.Checkbox(value=False, description='NovelAI')
m_naisfw = widgets.Checkbox(value=False, description='NovelAI SFW')
# SD 2
m_sd2 = widgets.Checkbox(value=False, description='Stable Diffusion 2.1')
m_sd2_768 = widgets.Checkbox(value=False, description='Stable Diffusion 2.1 768px')
m_sd2_depth = widgets.Checkbox(value=False, description='Stable Diffusion 2 Depth Model')
m_realengine = widgets.Checkbox(value=False, description='Realism Engine')
m_wd14 = widgets.Checkbox(value=False, description='Waifu Diffusion 1.4')
m_wd15_ink = widgets.Checkbox(value=False, description='Waifu Diffusion 1.5 Ink')
m_wd15_radiance = widgets.Checkbox(value=False, description='Waifu Diffusion 1.5 Radiance')
m_wd15_illusion = widgets.Checkbox(value=False, description='Waifu Diffusion 1.5 Illusion')
m_wd15_mofu = widgets.Checkbox(value=False, description='Waifu Diffusion 1.5 Mofu')
m_illumdif = widgets.Checkbox(value=False, description='Illuminutty Diffusion')
# SD XL
m_sdxl = widgets.Checkbox(value=False, description='Stable Diffusion XL')
m_sdxlrefiner = widgets.Checkbox(value=False, description='Stable Diffusion XL Refiner')
m_counterfeitxl = widgets.Checkbox(value=False, description='Counterfeit XL')
m_dreamshaperxl = widgets.Checkbox(value=False, description='Dreamshaper XL')
# Controlnet models
control_all = widgets.Checkbox(value=False, description='All')
control_canny = widgets.Checkbox(value=False, description='Canny')
control_mlsd = widgets.Checkbox(value=False, description='MLSD')
control_depth = widgets.Checkbox(value=False, description='Depth')
control_normal = widgets.Checkbox(value=False, description='Normal')
control_seg = widgets.Checkbox(value=False, description='Segmentation')
control_inpaint = widgets.Checkbox(value=False, description='Inpainting')
control_lineart = widgets.Checkbox(value=False, description='Lineart')
control_lineartanime = widgets.Checkbox(value=False, description='Lineart (anime)')
control_openpose = widgets.Checkbox(value=False, description='Openpose')
control_scribble = widgets.Checkbox(value=False, description='Scribble')
control_softedge = widgets.Checkbox(value=False, description='Soft Edge')
control_shuffle = widgets.Checkbox(value=False, description='Shuffle')
control_ip2p = widgets.Checkbox(value=False, description='Instruct Pix2Pix')
control_tile = widgets.Checkbox(value=False, description='Tile')

sd15_modellist = [m_sd15, m_sd15inpaint, m_realv13, m_realv2, m_chilloutmix, m_deliberate,
                 m_dreamshaperv3, m_dreamshaperv4, m_dreamshaperv5, m_ned, m_openjourney, m_rev,
                 m_perfectworld, m_wd13, m_anythingv3, m_anythingv4, m_anythingv5, m_counterfeitv2,
                 m_counterfeitv3, m_aom2, m_aom3, m_aom3a1, m_aom3a1b, m_aom3a2, m_aom3a3, m_bom,
                 m_midnightmaple, m_midnightmelt, m_midnightmixermelt, m_cornflowerv9, m_cornflowerx,
                 m_ambientmix, m_hassaku, m_grapefruit, m_meinamix, m_meinahentai, m_nai, m_naisfw]
sd2_modellist = [m_sd2, m_sd2_768, m_sd2_depth, m_illumdif, m_realengine, m_wd14,
                 m_wd15_radiance, m_wd15_ink, m_wd15_mofu, m_wd15_illusion]
sdxl_modellist = [m_sdxl, m_sdxlrefiner, m_counterfeitxl, m_dreamshaperxl]
all_modellist = sd15_modellist + sd2_modellist + sdxl_modellist
control_modellist = [
    control_canny, control_mlsd, control_depth, control_normal, control_seg, control_inpaint,
    control_lineart, control_lineartanime, control_openpose, control_scribble, control_softedge,
    control_shuffle, control_ip2p, control_tile
]

m_presets = {
    "None": [],
    "Stable Diffusion 1.5": [m_sd15, m_sd15inpaint],
    "Stable Diffusion 2": [m_sd2, m_sd2_768, m_sd2_depth],
    "Anything Model": [m_anythingv3, m_anythingv4, m_anythingv5],
    "Orange Mixes": [m_aom2, m_aom3, m_aom3a1, m_aom3a1b, m_aom3a2, m_aom3a3],
    "Midnight Mixes": [m_midnightmaple, m_midnightmelt, m_midnightmixermelt],
    "Waifu Diffusion 1.5": [m_wd15_radiance, m_wd15_ink, m_wd15_mofu, m_wd15_illusion]
}

m_presets_dropdown = widgets.Dropdown(
    options=list(m_presets.keys()),
    value='None',
    description='Preset:',
    disabled=False,
)

sd15_checkboxes = widgets.GridBox(
    sd15_modellist,
    layout=widgets.Layout(grid_template_columns="repeat(5, 250px)")
)
sd2_checkboxes = widgets.GridBox(
    sd2_modellist,
    layout=widgets.Layout(grid_template_columns="repeat(5, 250px)")
)
sdxl_checkboxes = widgets.GridBox(
    sdxl_modellist,
    layout=widgets.Layout(grid_template_columns="repeat(5, 250px)")
)
control_checkboxes = widgets.GridBox(
    [control_all] + control_modellist,
    layout=widgets.Layout(grid_template_columns="repeat(5, 250px)")
)

m_urldownloadstring = widgets.Textarea(
    value="",
    placeholder="Insert URLs here",
    description="Model URLs:",
    layout=widgets.Layout(width='1500px')
)
m_hftokenstring = widgets.Text(
    value="",
    placeholder="Insert huggingface token",
    description="HF Token:",
    layout=widgets.Layout(width='400px')
)

downloadmodel_button = widgets.Button(
    description='Download',
    disabled=False,
    button_style=''
)

m_downloadhint = """
Check the models you want to download below, or select a preset.
"""
m_downloadhint = m_downloadhint.strip().replace("\n", " ")
m_urlhint = """
Alternatively, fill this with download links (ex: <a href="https://huggingface.co/" target="_blank">huggingface</a>, <a href="https://civitai.com/" target="_blank">civitai</a>).
Download multiple models by separating each link with commas
<br>Insert your huggingface token if you need to download a private model
"""
m_urlhint = m_urlhint.strip().replace("\n", " ")

layout = widgets.VBox([
    widgets.HTML(value=m_downloadhint),
    m_presets_dropdown,
    widgets.HTML(value="<h2>SD 1.5</h2>"),
    sd15_checkboxes,
    widgets.HTML(value="<h2>SD 2</h2>"),
    sd2_checkboxes,
    widgets.HTML(value="<h2>SD XL</h2>"),
    sdxl_checkboxes,
    widgets.HTML(value=m_urlhint),
    m_urldownloadstring,
    m_hftokenstring,
    widgets.HTML(value="<h2>ControlNet Models</h2>"),
    control_checkboxes
])

# URL format: (url, filename, hf_token)
model_links = {
    m_sd15: ("https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt",),
    m_sd15inpaint: ("https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt",),
    m_realv13: ("https://civitai.com/api/download/models/6987",),
    m_realv2: ("https://civitai.com/api/download/models/29460",),
    m_chilloutmix: ("https://civitai.com/api/download/models/11732",),
    m_deliberate: ("https://civitai.com/api/download/models/15236",),
    m_dreamshaperv3: ("https://civitai.com/api/download/models/6202",),
    m_dreamshaperv4: ("https://civitai.com/api/download/models/25067",),
    m_dreamshaperv5: ("https://civitai.com/api/download/models/43888",),
    m_ned: ("https://civitai.com/api/download/models/11925",),
    m_openjourney: ("https://huggingface.co/prompthero/openjourney-v4/resolve/main/openjourney-v4.ckpt",),
    m_rev: ("https://civitai.com/api/download/models/46846",),
    m_perfectworld: ("https://civitai.com/api/download/models/77276",),
    m_wd13: ("https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float16.ckpt",),
    m_anythingv3: ("https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors",),
    m_anythingv4: ("https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned-fp16.ckpt",),
    m_anythingv5: ("https://civitai.com/api/download/models/30163",),
    m_counterfeitv2: ("https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5_fp16.safetensors",),
    m_counterfeitv3: ("https://huggingface.co/gsdf/Counterfeit-V3.0/resolve/main/Counterfeit-V3.0_fp16.safetensors"),
    m_aom2: ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/Pruned/AbyssOrangeMix2_sfw_pruned_fp16_with_VAE.safetensors",),
    m_aom3: ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3_orangemixs.safetensors",),
    m_aom3a1: ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors",),
    m_aom3a1b: ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1B_orangemixs.safetensors",),
    m_aom3a2: ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A2_orangemixs.safetensors",),
    m_aom3a3: ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors",),
    m_bom: ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/BloodOrangeMix/BloodOrangeMix.safetensors",),
    m_midnightmaple: ("https://huggingface.co/DrBob2142/Midnight_Mixes/resolve/main/Midnight%20Maple.safetensors",),
    m_midnightmelt: ("https://huggingface.co/DrBob2142/Midnight_Mixes/resolve/main/Midnight%20Melt.safetensors",),
    m_midnightmixermelt: ("https://huggingface.co/DrBob2142/Midnight_Mixes/resolve/main/Midnight%20Mixer%20Melt.safetensors",),
    m_cornflowerv9: ("https://civitai.com/api/download/models/11690",),
    m_cornflowerx: ("https://civitai.com/api/download/models/39123",),
    m_ambientmix: ("https://civitai.com/api/download/models/31866",),
    m_hassaku: ("https://civitai.com/api/download/models/62528",),
    m_grapefruit: ("https://civitai.com/api/download/models/29179",),
    m_meinamix: ("https://civitai.com/api/download/models/80511",),
    m_meinahentai: ("https://civitai.com/api/download/models/64046",),
    m_nai: ("https://huggingface.co/Chronophobia/animefull/resolve/main/nai-fp16.safetensors",),
    m_naisfw: ("https://huggingface.co/Chronophobia/animefull/resolve/main/naisfw-fp16.safetensors",),
    m_sd2: ("https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt", ("https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml", "v2-1_512-ema-pruned.yaml")),
    m_sd2_768: ("https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt", ("https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml", "v2-1_768-ema-pruned.yaml")),
    m_sd2_depth: ("https://huggingface.co/stabilityai/stable-diffusion-2-depth/resolve/main/512-depth-ema.ckpt", ("https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-midas-inference.yaml", "512-depth-ema.yaml")),
    m_realengine: ("https://civitai.com/api/download/models/20414", "https://civitai.com/api/download/models/20414?type=Config"),
    m_illumdif: ("https://civitai.com/api/download/models/42304","https://civitai.com/api/download/models/42304?type=Config"),
    m_wd14: ("https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/wd-1-4-anime_e1.ckpt", "https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/wd-1-4-anime_e1.yaml"),
    m_wd15_ink: ("https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-ink-fp16.safetensors",),
    m_wd15_illusion: ("https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-illusion-fp16.safetensors",),
    m_wd15_radiance: ("https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-radiance-fp16.safetensors",),
    m_wd15_mofu: ("https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-mofu-fp16.safetensors",),
    m_sdxl: ("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors",),
    m_sdxlrefiner: ("https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors"),
    m_counterfeitxl: (("https://huggingface.co/gsdf/CounterfeitXL/resolve/main/CounterfeitXL_%CE%B1.safetensors","CounterfeitXL.safetensors"),),
    m_dreamshaperxl: ("https://civitai.com/api/download/models/126688",)
}
controlnet_links = {
    control_canny: "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors",
    control_mlsd: "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors",
    control_depth: "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors",
    control_normal: "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors",
    control_seg: "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors",
    control_inpaint: "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors",
    control_lineart: "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors",
    control_lineartanime: "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors",
    control_openpose: "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors",
    control_scribble: "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors",
    control_softedge: "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors",
    control_shuffle: "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors",
    control_ip2p: "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors",
    control_tile: "https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors"
}

def selectModelPresets(change):
  selected_preset = change['new']
  checked_list = m_presets[selected_preset]
  for checkbox in all_modellist:
    if checkbox in checked_list:
      checkbox.value = True
    else:
      checkbox.value = False

def selectAllControl(change):
  check_bool = change['new']
  if check_bool:
    for checkbox in control_modellist:
      checkbox.value = True
      checkbox.disabled = True
  else:
    for checkbox in control_modellist:
      checkbox.value = False
      checkbox.disabled = False

def downloadModelURL(url, hf_token=None):
  filename = None
  if isinstance(url, (list, tuple)):
    model_url = url[0]
    try:
      filename = url[1]
    except IndexError:
      pass
    try:
      hf_token = url[2]
    except IndexError:
      pass
  else:
    model_url = url
  if not filename:
    filename = model_url.split('/')[-1].replace('%20', '_') if "huggingface.co" in model_url else None

  if hf_token and "huggingface.co" in model_url:
    user_header = f'"Authorization: Bearer {hf_token}"'
    auth = f'--header={user_header}'
  else:
    auth = ''
  if filename:
    output = f'-o {filename}'
  else:
    output = ''

  !aria2c --console-log-level=error {auth} -c -x 16 -s 16 -k 1M {model_url} {output}

def downloadModels():
  m_downloadlist = []
  m_hftoken = m_hftokenstring.value if m_hftokenstring.value else None
  for checkbox in all_modellist:
    if checkbox.value:
      m_downloadlist.extend(model_links[checkbox])
  if m_urldownloadstring.value:
    for model in m_urldownloadstring.value.strip().split(","):
      m_downloadlist.append(model)
  %cd {webui_dir}/models/checkpoints
  download_amount = len(m_downloadlist)
  download_left = 1
  for url in m_downloadlist:
    print(f"Downloading {download_left}/{download_amount}...")
    downloadModelURL(url, m_hftoken)
    download_left += 1
  %cd {webui_dir}/models/controlnet
  for model in control_modellist:
    if not model.value:
      continue
    else:
      url = controlnet_links[model]
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -o {url.split('/')[-1]}

control_all.observe(selectAllControl, names='value')
m_presets_dropdown.observe(selectModelPresets, names='value')

display(layout)

In [ ]:
#@title ## Download Checkpoints
#@markdown Run this cell once you configured the above cell.

vae_list = [
    ("https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt", "vae-ft-mse-840000-ema-pruned.ckpt"),
    ("https://huggingface.co/Chronophobia/animefull/resolve/main/animevae.pt", "animevae.pt"),
    ("https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime.ckpt", "kl-f8-anime.ckpt"),
    ("https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt", "kl-f8-anime2.ckpt")
]

downloadModels()

def downloadVAE(vae):
  url = vae[0]
  filename = vae[1]
  if not os.path.exists(f"{webui_dir}/models/vae/{filename}"):
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {webui_dir}/models/vae -o {filename}

for vae in vae_list:
  downloadVAE(vae)

In [ ]:
#@title ## Configure Custom Nodes
#@markdown Run this cell and follow further instructions
import ipywidgets as widgets
from IPython.display import display

# Extensions
cn_manager = widgets.Checkbox(value=True, description="ComfyUI Manager")
cn_advclip = widgets.Checkbox(value=True, description="Advanced CLIP Text Encode")
cn_efficiency = widgets.Checkbox(value=True, description="Efficiency Nodes")
cn_controlnet = widgets.Checkbox(value=False, description="ControlNet Preprocessors")
cn_cutoff = widgets.Checkbox(value=False, description="Cutoff")
cn_wasnode = widgets.Checkbox(value=False, description="WAS Node Suite")
cn_imagesgrid = widgets.Checkbox(value=False, description="ImagesGrid")
cn_sdupscale = widgets.Checkbox(value=False, description="Ultimate SD Upscale")

cn_list = [
    cn_manager, cn_advclip, cn_efficiency, cn_controlnet, cn_cutoff, cn_wasnode,
    cn_imagesgrid, cn_sdupscale
]
cn_checkboxes = widgets.GridBox(cn_list, layout=widgets.Layout(grid_template_columns="repeat(6, 250px)"))

cn_presets = {
    "None": [],
    "Minimum": [cn_manager, cn_advclip, cn_efficiency],
    "All": cn_list
}

cn_presets_dropdown = widgets.Dropdown(
    options=list(cn_presets.keys()),
    value="Minimum",
    description="Presets"
)

installcn_button = widgets.Button(
    description='Install',
    disabled=False,
    button_style=''
)

cn_layout = widgets.VBox([
    widgets.HTML(value="Check the nodes you want to install, or select a preset"),
    cn_presets_dropdown,
    cn_checkboxes
])

cn_links = {
    cn_manager: "ltdrdata/ComfyUI-Manager",
    cn_advclip: "BlenderNeko/ComfyUI_ADV_CLIP_emb",
    cn_controlnet: "Fannovel16/comfy_controlnet_preprocessors",
    cn_cutoff: "BlenderNeko/ComfyUI_Cutoff",
    cn_wasnode: "WASasquatch/was-node-suite-comfyui",
    cn_imagesgrid: "LEv145/images-grid-comfy-plugin",
    cn_sdupscale: "ssitu/ComfyUI_UltimateSDUpscale",
    cn_efficiency: "LucianoCirino/efficiency-nodes-comfyui"
}

def selectCNPresets(change):
  selected_preset = change['new']
  checked_list = cn_presets[selected_preset]
  for node in cn_list:
    if node in checked_list:
      node.value = True
    else:
      node.value = False

def cloneCNRepo(ext, ext_displayname=None):
  ext = ext.replace(";", "|")
  ext_commit = None
  ext_link, _, ext_commit = ext.strip().partition("|")
  if not ext_link.startswith("https://") and not ext_link.startswith("http://"):
    if not "github.com" in ext_link:
      ext_link = "github.com/" + ext_link
    ext_link = "https://" + ext_link
  ext_link = ext_link.replace("http://", "https://")
  if not ext_link.endswith('.git'):
    ext_link += '.git'
  ext_name = ext_link.split("/")[-1].split(".")[0]

  print(f"Installing {ext_displayname}...") if ext_displayname else print(f"Installing {ext_name}...")
  !git clone {ext_link} --recursive
  print(ext_name, "Install completed.")

def installCustomNodes():
  cn_install = []
  for node in cn_list:
    if node.value:
      cn_install.append((cn_links[node], node.description))
  for node in cn_install:
    %cd {webui_dir}/custom_nodes
    cloneCNRepo(node[0], node[1])
    if node[1] == "ControlNet Preprocessors":
      %cd comfy_controlnet_preprocessors
      !python install.py

cn_presets_dropdown.observe(selectCNPresets, names='value')
display(cn_layout)

In [ ]:
#@title ## Install Custom Nodes
#@markdown Run this cell once you configured the above cell.

installCustomNodes()

In [ ]:
#@title ## Run ComfyUI
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

%cd {webui_dir}

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server